In [1]:
from random import randint
from string import Template
import json

from cog.torque import Graph
import pygraphviz as pgv
from IPython.display import Image

In [2]:
def all_subclasses(cls):
    return list(set(cls.__subclasses__()).union(
        [s for c in cls.__sublcasses__() for s in all_subclasses(c)]))

In [3]:
class MetaSystem:
    graph = Graph("DMA")
    t = 0
    
    @classmethod
    def system(cls, name):
        for system in cls.HyperSystem.all_hypersystems():
            if system.__name__ == name:
                return system
        
        for system in cls.System.all_systems():
            if system.__name__ == name:
                return system

        raise Exception(f"Unknown system: {name}")
    
    class System:
        edge_name = "node"
        
        @classmethod
        def all_systems(cls):
            for subclass in cls.__subclasses__():
                yield from subclass.all_systems()
                yield subclass
        
        @classmethod
        def resolver(cls, entity_name, relationships):
            if self.edge_name in relationships:
                MetaSystem.put(entity_name, self.edge_name, relationships[self.edge_name])
                return self.edge_name
        
        @classmethod
        def relationships(cls):
            return MetaSystem.v().tag(cls.edge_name).inc(cls.edge_name).all()['result']
        
        @classmethod
        def relationships_by_id(cls):
            return {rel['id']: rel for rel in MetaSystem.v().tag(cls.edge_name).inc(cls.edge_name).all()['result']}
        
        @classmethod
        def validate(cls):
            pass
        
        @classmethod
        def get_relationship(cls, node_id):
            try:
                return MetaSystem.v(node_id).out(cls.edge_name).all()['result'][0]['id']
            except IndexError:
                raise Exception(f"{cls.__name__} relationship not found on {node_id}")
        
    class HyperSystem:
        systems = []
        
        @classmethod
        def all_hypersystems(cls):
            for subclass in cls.__subclasses__():
                yield from subclass.all_hypersystems()
                yield subclass
        
        @classmethod
        def get_edge_names(cls):
            return [system.edge_name for system in cls.systems]
        
        @classmethod
        def validate(cls):
            [system.validate() for system in cls.systems]
    
    @classmethod
    def reset(cls):
        !rm -rf /tmp/cog_home
        cls.graph = Graph("DMA")
    
    @classmethod
    def whole_graph(cls):
        return cls.graph.v().tag("from").out().tag("to")
    
    @classmethod
    def add_entity(cls, entity_name, **relationships):
        for rel_name, rel_value in relationships.items():
            cls.graph.put(entity_name, rel_name, rel_value)
    
    @classmethod
    def entity_exists(cls, entity_name):
        return (cls.v(entity_name).inc().count() + cls.v(entity_name).out().count()) > 0
    
    @classmethod
    def v(cls, *args, **kwargs):
        return cls.graph.v(*args, **kwargs)
    
    @classmethod
    def put(cls, entity_name, edge_name, system_value):
        cls.graph.put(entity_name, edge_name, system_value)
    
    @classmethod
    def validate(cls):
        results = {}
        for system in list(cls.System.all_systems()) + list(cls.HyperSystem.all_hypersystems()):
            try:
                system.validate()
            except Exception as exc:
                    results[system.__name__] = exc
        return results
    
    @classmethod
    def trace_edges(cls, edge_name, g=None):
        g = cls.v("output") if g is None else g
        edges = g.inc(edge_name).all()['result']
        return [f['id'] for f in edges] + (trace_edges(g) if len(edges) > 0 else [])
    
    @staticmethod
    def path_dir(graph, reverse=True):
        return graph.inc if reverse else graph.out
    
    @classmethod
    def build_traversal_at_depth(cls, edge_name, depth, start=None, reverse=True):
        g = cls.v() if start is None else cls.v(start)
        for i in range(depth):
            g = cls.path_dir(g.tag(f"depth{i}"), reverse)(edge_name).tag("end")
        return g
    
    @classmethod
    def get_paths(cls, edge_name, end=None, start=None, reverse=True):
        g = cls.v(start) if start is not None else cls.v()
        depth = 0
        all_paths = []
        while (g := cls.build_traversal_at_depth(edge_name, depth, start=start, reverse=reverse)).count() > 0:
            for path in g.all()['result']:
                if pathDir(cls.v(path['id']), reverse)(edge_name).count() == 0:
                    all_paths.append(path)
            depth += 1
        return all_paths
    
    @classmethod
    def render(cls):
        if pgv is None:
            raise Exception("Missing graphviz requirement")
        
        def color_selector(value, color_set='node'):
            edge_set = ["black", "royalblue1", "slateblue", "coral"]
            node_set = ["red", "webmaroon", "green", "chartreuse4", "aquamarine2"]
            color_set = node_set if color_set == 'node' else edge_set
            try:
                float(value)
                return "purple"
            except ValueError:
                pass
            
            if value not in color_selector.color_map:
                color_selector.color_map[value] = color_set[randint(0, len(color_set) - 1)]

            return color_selector.color_map[value]
        
        color_selector.color_map = {}
        
        vg = pgv.AGraph()
        nodes = []
        for rel in cls.whole_graph().all('e')['result']:
            if rel['to'] not in nodes:
                vg.add_node(rel['to'], color=color_selector(rel['to']))
                nodes.append(rel['to'])
            if rel['from'] not in nodes:
                vg.add_node(rel['from'], color=color_selector(rel['from']))
                nodes.append(rel['from'])
            vg.add_edge(rel['to'], rel['from'], label=rel['edges'][0],
                        color=color_selector(rel['edges'][0], color_set="edge"),
                        len=2)
        
        vg.layout(prog='neato')
        vg.draw('DMA.png')
        return Image('DMA.png')
ms = MetaSystem

In [4]:
class AeonicMetaSystem(MetaSystem):
    aeon = 0
    aeon_edge_name = "aeon"
    
    @classmethod
    def put(cls, entity_name, edge_name, system_value):
        cls.graph.put(entity_name, edge_name, system_value)
        cls.graph.put(system_value, cls.aeon_edge_name, cls.aeon)
    
    @classmethod
    def observe(cls):
        aeon += 1

In [14]:
hex(5 + 1)

'0x6'